In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Conv1D, MaxPooling1D

In [3]:
import json
with open('data.json', 'r') as f:
    labels = json.load(f)
labels

{'Boy': 0,
 'Can': 1,
 'Eat': 2,
 'Fine': 3,
 'Girl': 4,
 'Help': 5,
 'How': 6,
 'Hungry': 7,
 'I': 8,
 'Mother': 9,
 'Namaste': 10,
 'Name': 11,
 'Parents': 12,
 'Sister': 13,
 'Sleep': 14,
 'This': 15,
 'You': 16}

In [4]:
import json
with open('landmarks.json', 'r') as f:
    landmarks = json.load(f)
landmarks.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

In [5]:
len(landmarks['0'][0])

4356

In [6]:
with open('length.json', 'r') as f:
    length = json.load(f)
length.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', 

In [7]:
max_length=max(length.values())
max_length

5940

In [46]:
Y=pd.read_csv('labels.csv')
Y.tail()

,Class Label
717,16
718,16
719,16
720,16
721,16


In [9]:
f=open('words.txt','r')
words=f.read().split('\n')
words

['Boy',
 'Can',
 'Eat',
 'Fine',
 'Girl',
 'Help',
 'How',
 'Hungry',
 'I',
 'Mother',
 'Namaste',
 'Name',
 'Parents',
 'Sister',
 'Sleep',
 'This',
 'You',
 '']

In [87]:
# X = pd.DataFrame(columns=['Landmarks'])
X=[]

In [88]:
def pad_data():
    for i in landmarks:
        landmarks[i][0].extend([1]*(max_length-len(landmarks[i][0])))
        X.append(np.array(landmarks[i]))

In [89]:
pad_data()

In [90]:
len(X)

722

In [91]:
X

[array([[-0.2904878 , -0.16641041, -0.09852826, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1.14277988, -1.14277988, -1.14277988, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1.06837073, -1.06837073, -1.06837073, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-0.22918894, -0.11131902, -0.05098939, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-0.87611416, -0.80004271, -0.7496262 , ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1.12677718, -1.12677718, -1.12677718, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1.13677586, -1.13677586, -1.13677586, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-0.73616662, -0.6641765 , -0.61268805, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-0.17041456, -0.12579402, -0.10721782, ...,  0.        ,
          0.        ,  0.        ]]),
 array([[-1.10825992, -1.10825992, -1.10825992, ...,  0.        

In [93]:
X=np.array(X)
X.shape

(722, 1, 5940)

In [47]:
Y=to_categorical(Y).astype(int)
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [17]:
Y.shape

(722, 17)

In [95]:
X.shape

(722, 1, 5940)

In [96]:
#X has to be of Series
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [97]:
x_train.shape

(577, 1, 5940)

In [98]:
x_test.shape

(145, 1, 5940)

In [21]:
type(x_train)

numpy.ndarray

In [99]:
x_train.shape

(577, 1, 5940)

In [23]:
y_train.shape

(577, 17)

In [100]:
import os
log_dir=os.path.join('logs')
tb_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir)


Model-1

In [101]:
model=Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(62, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(words)-1, activation='softmax'))

In [102]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [103]:
model.fit(x_train, y_train, epochs=1000, batch_size=32, validation_data=(x_test, y_test), callbacks=[tb_callback])

Epoch 1/1000
19/19 [==============================] - 8s 64ms/step - loss: 6.9366 - categorical_accuracy: 0.0728 - val_loss: 4.9213 - val_categorical_accuracy: 0.0414
Epoch 2/1000
19/19 [==============================] - 1s 29ms/step - loss: 8.4592 - categorical_accuracy: 0.0624 - val_loss: 3.9288 - val_categorical_accuracy: 0.0414
Epoch 3/1000
19/19 [==============================] - 1s 29ms/step - loss: 8.4880 - categorical_accuracy: 0.0797 - val_loss: 4.4489 - val_categorical_accuracy: 0.0897
Epoch 4/1000
19/19 [==============================] - 1s 29ms/step - loss: 5.3072 - categorical_accuracy: 0.0849 - val_loss: 4.7212 - val_categorical_accuracy: 0.0138
Epoch 5/1000
19/19 [==============================] - 1s 28ms/step - loss: 5.2485 - categorical_accuracy: 0.1057 - val_loss: 3.8629 - val_categorical_accuracy: 0.0552
Epoch 6/1000
19/19 [==============================] - 1s 29ms/step - loss: 4.6319 - categorical_accuracy: 0.0971 - val_loss: 4.1185 - val_categorical_accuracy: 0.020

KeyboardInterrupt: 

In [150]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 1, 64)             1537280   
                                                                 
 lstm_10 (LSTM)              (None, 1, 128)            98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 62)                4030      
                                                                 
 dense_10 (Dense)            (None, 32)                2016      
                                                                 
 dense_11 (Dense)            (None, 18)                594       
                                                                 
Total params: 1692144 (6.46 MB)
Trainable params: 1692

In [ ]:
res[0]

MODEL 2

In [107]:
model=Sequential()
model.add(Conv1D(16, kernel_size=3, activation='relu',input_shape=(X.shape[1], X.shape[2])))
# model.add(MaxPooling1D())

model.add(Conv1D(32, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D())

model.add(Conv1D(16, kernel_size=3, activation='relu'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(LSTM(128, return_sequences=True, activation='relu'))

model.add(Dense(256, activation='relu'))
model.add(Dense(len(words)-1, activation='softmax'))

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_2. Consider increasing the input size. Received input shape [None, 1, 5940] which would produce output shape with a zero or negative value in a dimension.

In [108]:
model = Sequential()
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], X.shape[2])))
# model.add(MaxPooling1D())

model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling1D())

model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(LSTM(128, return_sequences=True, activation='relu'))

model.add(Dense(256, activation='relu'))
model.add(Dense(len(words) - 1, activation='softmax'))


ValueError: Exception encountered when calling layer "max_pooling1d" (type MaxPooling1D).

Negative dimension size caused by subtracting 2 from 1 for '{{node max_pooling1d/MaxPool}} = MaxPool[T=DT_FLOAT, data_format="NHWC", explicit_paddings=[], ksize=[1, 2, 1, 1], padding="VALID", strides=[1, 2, 1, 1]](max_pooling1d/ExpandDims)' with input shapes: [?,1,1,16].

Call arguments received by layer "max_pooling1d" (type MaxPooling1D):
  • inputs=tf.Tensor(shape=(None, 1, 16), dtype=float32)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [ ]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[tb_callback])

In [ ]:
model.summary()

Model 3

In [ ]:
model=Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
model.add(MaxPooling2D())

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(len(words), activation='softmax'))

In [ ]:
res=model.predict(x_test)
res[0]

##Save model

In [ ]:
model.save('model.h5')

model=tf.keras.models.load_model('model.h5')

Prediction

In [ ]:
words[np.argmax(model.predict(x_test)[0])]

In [ ]:
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
y_pred=model.predict(x_test)

In [ ]:
y_true=np.argmax(y_test, axis=1).tolist()
y_pred=np.argmax(y_pred, axis=1).tolist()

In [ ]:
y_true

In [ ]:
multilabel_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true, y_pred)